In [ ]:
import os
import re
import sys

import pathlib
from pathlib import Path

import sqlite3

import pandas as pd
import numpy as np

from scipy import stats

import math

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

import pickle
import xlrd
import pylab

In [ ]:
# Helper (support)
from support import input_helper

In [ ]:
# Directory with IGU datasets:
ROOT_DIR = "RawData"

# Specify path to dynamic stock model and to datafile, relative
MainPath = os.path.join('odym', 'modules')
DataPath = os.path.join('docs', 'files')
sys.path.insert(0, MainPath)  # add ODYM module directory to system path

# add ODYM module directory to system path, absolute
sys.path.insert(0, os.path.join(os.getcwd(), 'odym', 'modules'))
sys.path.insert(0, os.path.join(os.getcwd(), 'docs', 'files'))

# Define path to save figures:
path_img = (
    r'C:\Users\souvi\Documents\These\90_Presentations & Writting\50_Papers\2021_12_MEFA of IGU\IMG'
)

In [ ]:
import ODYM_Classes as msc  # import the ODYM class file
import ODYM_Functions as msf  # import the ODYM function file
import dynamic_stock_model as dsm  # import the dynamic stock model library

In [ ]:
# Define size of figure:
mpl.rcParams['figure.figsize'] = (16, 10)

In [ ]:
# Define seaborn main parameters:
sns.set_style("ticks")
sns.color_palette("colorblind")
sns.set_context("paper", font_scale=1.5,
                rc={"axes.titlesize": 15, "lines.linewidth": 1.2,
                    "legend.fontsize": 10, "legend.title_fontsize": 10})

In [ ]:
pd.set_option('precision', 2)

In [ ]:
# A custom formatter function which divide by 1000 an axis:
def yfmt1000(x, pos): 
    s = f'{x/1000:,.1f}'
    return s

# W/ classes for tick-locating and -formatting:
import matplotlib.ticker as tkr

# A formatter function:
y_1000 = tkr.FuncFormatter(yfmt1000)

# To divide the y-axis by 1000 when needed:
# ax.yaxis.set_major_formatter(y_1000)

In [ ]:
# A function used to define the thickness of x and y axis:
def style_ax(ax):
    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(0.5)
        ax.tick_params(width=0.5)
        ax.set_xlabel(None)
    return ax

In [ ]:
# A keyword to export figures, or not:
export = False

# Upload Datasets

In [ ]:
be_data = input_helper.get_data(
    'BE_IGU_RawData_VPython.xlsx', directory=ROOT_DIR)
fr_data = input_helper.get_data(
    'FR_IGU_RawData_VPython.xlsx', directory=ROOT_DIR)
eu_data = input_helper.get_data(
    'eu_igu_RawData_VPython.xlsx', directory=ROOT_DIR)

In [ ]:
print("be_data, sheet names = \n {}\n".format(be_data.sheet_names))
print("fr_data, sheet names = \n {}\n".format(fr_data.sheet_names))
print("eu_data, sheet names = \n {}\n".format(eu_data.sheet_names))

# Material, Energy and Carbon Intensity in the Flat Glass Industry

See: Souviron and Khan. (Forthcoming). "The Materiality of Transparency: An Environmental Analysis of the Architectural Flat Glass Industry since 1945 in Europe, France and Belgium". Resources, Conservation and Recycling.

## Material intensity since 1945

In [ ]:
# Create a DataFrame for material intensity:
mat_intensity = fr_data.parse('RawMat_Intensity').set_index('year')

mat_intensity['cullet, kg/kg'] = (mat_intensity['internal cullet, kg/kg']
                                  + mat_intensity['external cullet, kg/kg'])

In [ ]:
# List of raw materials:
materials = ['sand', 'cullet', 'sodium carbonate',
             'limestone', 'dolomite', 'feldspar']

In [ ]:
# Plot raw materials intensity since 1945:
fig, axes = plt.subplots(nrows=2, ncols=3,
                         sharex=True, sharey=True,
                         figsize=(16, 5))

for i, (ax, mat) in enumerate(zip(axes.flatten(), materials)):
    ax.set_title(mat)
    (mat_intensity[f"{mat}, kg/kg"]
     .interpolate(method='pchip', limit_direction='forward')
     .plot(ax=ax)
     )

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 1)

fig.suptitle("Material intensity, per raw material [kg/kg]", y=1.05)
fig.subplots_adjust(hspace=0.4)

sns.despine(offset=5)
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3,
                         sharex=True, sharey=True,
                         figsize=(16, 5))

for i, (ax, mat) in enumerate(zip(axes.flatten(), materials)):
    ax.set_title(mat)
    (mat_intensity[f"{mat}, kg/kg"]
     .interpolate(method='pchip', limit_direction='forward')
     .rolling(5, center=True).mean()
     .plot(ax=ax))

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 1)

fig.suptitle("Material intensity, per raw material, 5-y rolled [kg/kg]",
             y=1.05)
fig.subplots_adjust(hspace=0.4)

sns.despine(offset=5)
plt.show()

In [ ]:
# Estimate the material intensity (sum of the intenisities per material):
mat_intensity['Total raw materials, kg/kg'] = 0

for mat in materials:
    mat_intensity['Total raw materials, kg/kg'] = (
        mat_intensity[f"{mat}, kg/kg"]
        + mat_intensity['Total raw materials, kg/kg']
    )

In [ ]:
# Plot the material intensity:
fig, ax = plt.subplots(figsize=(8, 5))

sns.lineplot(data=mat_intensity,
             x=mat_intensity.index,
             y=(mat_intensity['Total raw materials, kg/kg']
                .interpolate(method='pchip', limit_direction='forward')),
             ax=ax)

ax.set_xlim(1940, 2020)
ax.set_ylim(0, 2)

fig.suptitle("Total intensity of raw materials [kg/kg]")

sns.despine(offset=5)
plt.show()

In [ ]:
# Plot a 5-y rolled material intensity:
fig, ax = plt.subplots(figsize=(8, 5))

sns.lineplot(data=mat_intensity,
             x=mat_intensity.index,
             y=(mat_intensity['Total raw materials, kg/kg']
                .interpolate(method='pchip', limit_direction='forward')
                 .rolling(5, center=True).mean()),
             ax=ax)

ax.set_xlim(1940, 2020)
ax.set_ylim(0, 2)

fig.suptitle("Total intensity of raw materials, 5-y. rolled [kg/kg]")

sns.despine(offset=5)
plt.show()

In [ ]:
matwocullet = ['sand', 'sodium carbonate', 'limestone',
               'dolomite', 'feldspar']

In [ ]:
# Estimate the material intensity without cullet:
mat_intensity['Total raw mat w/o cullet, kg/kg'] = 0

for mat in matwocullet:
    mat_intensity['Total raw mat w/o cullet, kg/kg'] = (
        mat_intensity[f"{mat}, kg/kg"]
        + mat_intensity['Total raw mat w/o cullet, kg/kg']
    )

In [ ]:
# Plot a 5-y rolled material intensity:
fig, ax = plt.subplots(figsize=(8, 5))

sns.lineplot(data=mat_intensity,
             x=mat_intensity.index,
             y=(mat_intensity['Total raw mat w/o cullet, kg/kg']
                .interpolate(method='pchip', limit_direction='forward')
                 .rolling(5, center=True).mean()),
             ax=ax, linestyle='--')

sns.lineplot(data=mat_intensity,
             x=mat_intensity.index,
             y=(mat_intensity['Total raw materials, kg/kg']
                .interpolate(method='pchip', limit_direction='forward')
                 .rolling(5, center=True).mean()),
             ax=ax)

ax.set_xlim(1940, 2020)
ax.set_ylim(0, 2)

fig.suptitle("Total intensity of raw mat. w/o cullet, 5-y. rolled [kg/kg]")

sns.despine(offset=5)
plt.show()

## Energy Intensity since 1945

In [ ]:
# Create a DataFrame for energy intensity:
energy_intensity = fr_data.parse('Energy_Intensity').set_index('year')

In [ ]:
energies = ['fuel oil', 'natural gas', 'electricity']

In [ ]:
# Plot the intensity per type of energy:
fig, axes = plt.subplots(nrows=1, ncols=3,
                         sharex=True, sharey=True,
                         figsize=(16, 4))

for i, (ax, energy) in enumerate(zip(axes.flatten(), energies)):
    ax.set_title(energy)
    (energy_intensity[f"{energy}, GJ/t"]  # Interpolate:
     .interpolate(method='pchip', limit_direction='forward')
     .plot(ax=ax))

ax.set_xlim(1950, 2010)
ax.set_ylim(ymin=0)

fig.suptitle("Intensity per type of energy [GJ/t]", y=1.05)

sns.despine(offset=5)
plt.show()

In [ ]:
# Estimate the energy intensity (sum of the intensities per type of energy):
energy_intensity['Total energy, GJ/t'] = 0

for energy in energies:
    energy_intensity['Total energy, GJ/t'] = (
        energy_intensity['Total energy, GJ/t']
        + energy_intensity[f"{energy}, GJ/t"]
    )

In [ ]:
fossilfuels = ['fuel oil', 'natural gas']

In [ ]:
# Estimate the energy intensity (sum of the intensities per type of energy):
energy_intensity['Total energy w/o elec., GJ/t'] = 0

for energy in fossilfuels:
    energy_intensity['Total energy w/o elec., GJ/t'] = (
        energy_intensity['Total energy w/o elec., GJ/t']
        + energy_intensity[f"{energy}, GJ/t"]
    )

In [ ]:
# Plot the total energy intensity:
fig, ax = plt.subplots(figsize=(8, 5))

sns.lineplot(data=energy_intensity,
             x=energy_intensity.index,
             y=(energy_intensity['Total energy, GJ/t']
                .interpolate(method='pchip', limit_direction='forward')),
             ax=ax, linestyle='--')

sns.lineplot(data=energy_intensity,
             x=energy_intensity.index,
             y=(energy_intensity['Total energy w/o elec., GJ/t']
                .interpolate(method='pchip', limit_direction='forward')),
             ax=ax)

ax.set_xlim(1950, 2010)
ax.set_ylim(0, 40)

fig.suptitle("Total energy intensity, fossil fuels only [GJ/t]")

sns.despine(offset=5)
plt.show()

##  Carbon Intensity of the Flat Glass Production Process

In [ ]:
# Create a DataFrame only with the CO2 emission data:
emissions = fr_data.parse('emissions').set_index('year')

co2_intensity = emissions[['CO2 glass ind, kg/t']].copy()
co2_intensity.rename(columns={'CO2 glass ind, kg/t': 'CO2 [kg/t]'},
                     inplace=True)

co2_intensity.dropna()

In [ ]:
# Interpolate co2 intensity accoring to a pchip method:
co2_intensity['co2 interpolated [kg/t]'] = (
    co2_intensity['CO2 [kg/t]'].interpolate('pchip')
)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

sns.lineplot(data=co2_intensity, palette="GnBu")

ax.scatter(x=co2_intensity.index, y=co2_intensity['CO2 [kg/t]'],
           s=20)

ax.set_xlim(1945, 2020)
ax.set_ylim(ymin=0)
fig.suptitle("co2 intensity of the French flat glass industry [kgCO2/t]")
sns.despine(offset=5)
plt.show()

## Summary

In [ ]:
# Plot the material instensity per raw material:
fig, axes = plt.subplots(nrows=2, ncols=3,
                         sharex=True, sharey=True,
                         figsize=(16, 5))

for i, (ax, mat) in enumerate(zip(axes.flatten(), materials)):
    (mat_intensity[f"{mat}, kg/kg"]
     .interpolate(method='pchip', limit_direction='forward')
     .rolling(5, center=True).mean()
     .plot(ax=ax, c='sandybrown', linewidth=2))

    ax.scatter(x=mat_intensity.index, y=mat_intensity[f"{mat}, kg/kg"],
               s=40, marker='x', c='grey')
    plt.xticks(np.arange(1950, 2021, 10), rotation=45)

    ax.set_title(f" {mat}", loc='left')
    ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
    style_ax(ax)

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 1)

ax.minorticks_off()
plt.xticks(np.arange(1950, 2021, 10))

fig.suptitle("(a) Intensity per raw material [t/t]", x=0.1, y=1, ha='left')
fig.subplots_adjust(wspace=0.2, hspace=0.4)

sns.despine(offset=5)

if export:
    # Save image:
    fig.savefig(os.path.join(path_img, 'Fig4_Mat Intensity.png'),
                dpi=600, bbox_inches='tight')
    fig.savefig(os.path.join(path_img, 'Fig4_Mat Intensity.pdf'),
                bbox_inches='tight')

plt.show()

In [ ]:
# Plot the energy intensity per source of energy:
fig, axes = plt.subplots(nrows=1, ncols=3,
                         sharex=True, sharey=True,
                         figsize=(16, 2.5)
                         )

for i, (ax, energy) in enumerate(zip(axes.flatten(), energies)):
    (energy_intensity[f"{energy}, GJ/t"]
     .interpolate(method='pchip', limit_direction='forward')
     .plot(ax=ax, color='steelblue', linewidth=2))

    ax.scatter(x=energy_intensity.index, y=energy_intensity[f"{energy}, GJ/t"],
               s=40, marker='x', c='grey')

    ax.set_title(f" {energy}", loc='left')
    ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
    style_ax(ax)

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 40)

fig.suptitle("(b) Intensity per energy source [GJ/t]",
             x=0.1, y=1.12, ha='left')
fig.subplots_adjust(wspace=0.2)

ax.minorticks_off()
plt.xticks(np.arange(1950, 2021, 10))

sns.despine(offset=5)

if export:
    # Save image:
    fig.savefig(os.path.join(path_img, 'Fig4_Energy Intensity.png'),
                dpi=600, bbox_inches='tight')
    fig.savefig(os.path.join(path_img, 'Fig4_Energy Intensity.pdf'),
                bbox_inches='tight')

plt.show()

In [ ]:
# Plot the intensity for energy, material and CO2 emissions:
fig, axes = plt.subplots(nrows=1, ncols=3,
                         sharex=True,
                         figsize=(16, 2.5))

# Material intensity
ax = axes[0]
(mat_intensity['Total raw materials, kg/kg']
 .interpolate(method='pchip', limit_direction='forward')
 .rolling(5, center=True).mean()
 .plot(ax=ax, c='firebrick', linestyle='--', linewidth=1.25)
 )

(mat_intensity['Total raw mat w/o cullet, kg/kg']
 .interpolate(method='pchip', limit_direction='forward')
 .rolling(5, center=True).mean()
 .plot(ax=ax, c='firebrick', linestyle='-', linewidth=2)
 )

ax.scatter(x=mat_intensity.index,
           y=mat_intensity['Total raw mat w/o cullet, kg/kg'],
           s=40, marker='x', c='grey')

ax.set_title("Material intensity [t/t]", loc='left')
ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
style_ax(ax)
ax.set_ylim(0, 2)
ax.set_xlim(1950, 2020)

# Energy intensity
ax = axes[1]
(energy_intensity['Total energy, GJ/t']
 .interpolate(method='pchip', limit_direction='forward')
 .rolling(5, center=True).mean()
 .plot(ax=ax, c='firebrick', linestyle='--', linewidth=1.25))

(energy_intensity['Total energy w/o elec., GJ/t']
 .interpolate(method='pchip', limit_direction='forward')
 .rolling(5, center=True).mean()
 .plot(ax=ax, c='firebrick', linewidth=2))

ax.scatter(x=energy_intensity.index,
           y=energy_intensity['Total energy w/o elec., GJ/t'],
           s=40, marker='x', c='grey')

ax.set_title("Energy intensity [GJ/t]", loc='left')
ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
style_ax(ax)
ax.set_ylim(0, 40)

# Carbon dioxide intensity
ax = axes[2]
(co2_intensity['co2 interpolated [kg/t]'].rolling(5, center=True).mean()
 .plot(ax=ax, c='firebrick', linewidth=2))

ax.scatter(x=co2_intensity.index, y=co2_intensity['CO2 [kg/t]'],
           s=40, marker='x', c='grey')

ax.set_title("co2 intensity [kgCO2/t]", loc='left')
ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
style_ax(ax)
ax.set_ylim(0, 2500)

ax.minorticks_off()
plt.xticks(np.arange(1950, 2021, 10))

fig.suptitle("(c) Intensity, total", x=0.1, y=1.12, ha='left')
fig.subplots_adjust(wspace=0.20)

sns.despine(offset=5)

if export:
    # Save image:
    plt.savefig(os.path.join(path_img, 'Fig4_TotalIntensity.png'),
                dpi=600, bbox_inches='tight')
    plt.savefig(os.path.join(path_img, 'Fig4_TotalIntensity.pdf'),
                bbox_inches='tight')

plt.show()

# Flows of Insulating Glass Units in Europe

## Creating a Dataframe for the Flows of IGU

In [ ]:
# Data relating to IGU production:
eu_prod = eu_data.parse('prod').set_index(['Nbr of countries', 'year'])

In [ ]:
# Add import and export flows:
eu_imp = eu_data.parse('import').set_index(['Nbr of countries', 'year'])
eu_exp = eu_data.parse('export').set_index(['Nbr of countries', 'year'])

In [ ]:
# Create a new DataFrame for European IGU flows:
eu_igu = pd.concat([eu_prod['IGU, "000 m²']/1000,
                    eu_imp['IGU, "000 m²']/1000,
                    eu_exp['IGU, "000 m²']/1000],
                   axis=1, keys=[('Production', '[10^6 m²]'),
                                 ('Import', '[10^6 m²]'),
                                 ('Export', '[10^6 m²]')],
                   names=['Info', 'Unit']).stack(dropna=False)

eu_igu = eu_igu.unstack()

In [ ]:
eu_igu.style

In [ ]:
# Adding columns for data in [kt],
# to be used later for material/energy/carbon intensity
# 2500kg/m³ and in average 9mm thick

eu_igu[('Production', '[kt]')] = eu_igu[('Production', '[10^6 m²]')] * 2.5 * 9
eu_igu[('Import', '[kt]')] = eu_igu[('Import', '[10^6 m²]')] * 2.5 * 9
eu_igu[('Export', '[kt]')] = eu_igu[('Export', '[10^6 m²]')] * 2.5 * 9

eu_igu = eu_igu.sort_index(axis=1)

In [ ]:
eu_igu

## Production of Insulating Glass Units

In [ ]:
# Plot the European production of flat glass
fig, ax = plt.subplots()

sns.scatterplot(data=eu_igu['Production'].reset_index(),
                x='year', y='[10^6 m²]',
                hue='Nbr of countries',
                palette='colorblind',
                ax=ax)

sns.lineplot(data=eu_igu['Production'].reset_index(),
             x='year', y='[10^6 m²]',
             hue='Nbr of countries',
             palette='colorblind',
             ax=ax)

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 160)

ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
style_ax(ax)

fig.suptitle('European production of IGU [million m²]', fontsize=13)
sns.despine(offset=5)
plt.show()

In [ ]:
# Plot the European production of flat glass
fig, ax = plt.subplots()

sns.scatterplot(data=eu_igu['Production'].reset_index(),
                x='year', y='[10^6 m²]',
                hue='Nbr of countries',
                palette='colorblind',
                ax=ax)

sns.lineplot(data=(eu_igu['Production']
                   .interpolate(method='linear').reset_index()),
             x='year', y='[10^6 m²]',
             hue='Nbr of countries',
             palette='colorblind',
             ax=ax)

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 160)

ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
style_ax(ax)

fig.suptitle('European production of IGU [million m²]', fontsize=13)
sns.despine(offset=5)
plt.show()

## Consumption of Insulating Glass Units

In [ ]:
# Estimate the consumption of IGU in the EU:
eu_igu = eu_igu.stack(dropna=False)

eu_igu['Consumption'] = (eu_igu['Production']
                         + eu_igu['Import']
                         - eu_igu['Export']
                         )

eu_igu = eu_igu.unstack()

In [ ]:
# Plot the European consumption of IGU:
fig, ax = plt.subplots(figsize=(8, 5))

sns.lineplot(data=eu_igu['Consumption'].reset_index(),
             x='year', y='[10^6 m²]',
             hue='Nbr of countries',
             palette='colorblind',
             ax=ax)

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 160)

fig.suptitle('European consumption of IGU [million m²]')
sns.despine(offset=5)
plt.show()

## Summary

In [ ]:
# Key dates for the changes in the MFA scope, i.e. EU enlargement:
eu_kdates = {'EU6': 1972, 'EU10': 1980, 'EU12': 1986,
             'EU15': 1995, 'EU27': 2003, 'EU28': 2012}

In [ ]:
# Plot a synthesis of the results concerning European flows of IGU:
data1 = ['Production', 'Consumption']
data2 = ['Import', 'Export']

fig, axes = plt.subplots(nrows=2, ncols=2,
                         sharex=True, sharey=True,
                         figsize=(16, 10))

# Plot production and consumption:
for col, data in enumerate(data1):
    ax = axes[0][col]
    sns.lineplot(data=eu_igu[f"{data}"].reset_index(),
                 x='year', y="[10^6 m²]",
                 hue='Nbr of countries',
                 palette='colorblind', linewidth=1.75,
                 ax=ax)

    # Plot lines highlighting changes in the scope, i.e., EU enlargement:
    for y, x in eu_kdates.items():
        ax.axvline(x=x, c='grey', linestyle=':', linewidth=0.75)
        ax.text(x+0.5, 175, f"{y}", fontsize=13,
                verticalalignment='top')

    if col == 0:
        # Plot data gathered from literature review:
        sns.scatterplot(data=eu_igu[f"{data}"].reset_index(),
                        x='year', y="[10^6 m²]",
                        hue='Nbr of countries',
                        palette='colorblind',
                        marker='.', s=100,
                        ax=ax)

    ax.get_legend().remove()
    style_ax(ax)

    ax.set_title(f"{data}", pad=30, loc='left')
    #ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
    # ax.get_legend().remove()
    ax.yaxis.label.set_visible(False)

# Plot import and export below:
for col, data in enumerate(data2):
    ax = axes[1][col]
    sns.lineplot(data=eu_igu[f"{data}"].reset_index(),
                 x='year', y="[10^6 m²]",
                 hue='Nbr of countries',
                 palette='colorblind', linewidth=1.75,
                 ax=ax)

    # Plot lines highlighting changes in the scope, i.e., EU enlargement:
    for y, x in eu_kdates.items():
        ax.axvline(x=x, c='grey', linestyle=':', linewidth=0.75)
        ax.text(x+0.5, 175, f"{y}", fontsize=13,
                verticalalignment='top')

    style_ax(ax)

    ax.set_title(f"{data}", pad=20, loc='left')
    #ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
    ax.get_legend().remove()
    ax.yaxis.label.set_visible(False)


# Add legend:
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right', ncol=2,
           title='Nbr of countries:',
           bbox_to_anchor=(0.83, 0.98))

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 175)

fig.suptitle("European flows of flat glass [10^6 m²]", y=1.05)
fig.subplots_adjust(wspace=0.25, hspace=0.35)

sns.despine(offset=5)
plt.show()

if export:
    # Save image:
    fig.savefig(os.path.join(path_img, 'AppendixB_eu_iguFlows.png'),
                dpi=600, bbox_inches='tight')
    fig.savefig(os.path.join(path_img, 'AppendixB_eu_iguFlows.pdf'),
                bbox_inches='tight')

Hypothesis for missing data: EU production = EU consumption

In [ ]:
eu_igu = eu_igu.stack(dropna=False)
eu_igu['Estimated consumption'] = (eu_igu['Consumption']
                                   .fillna(eu_igu['Production']))

eu_igu = eu_igu.unstack()

In [ ]:
# Plot two figures:
fig, axes = plt.subplots(nrows=1, ncols=2,
                         sharex=True, sharey=True,
                         figsize=(16, 5))

# Number of countries for trend curves:
eu_list = [(6, 12), (15, 15), (27, 28)]

for col, data in enumerate(['Production', 'Estimated consumption']):
    for i, j in eu_list:
        ax = axes[col]
        # Plot the trend curve for flat glass production:
        ax.plot(eu_igu.interpolate().rolling(5, center=True).mean().loc[i:j]
                .reset_index(level='Nbr of countries')
                .sort_values(by=['year'])
                [(f"{data}", "[10^6 m²]")],
                c='black', linestyle='--')

        if col == 0:
            # Plot data gathered from literature review:
            sns.scatterplot(data=eu_igu["Production"].reset_index(),
                            x='year', y="[10^6 m²]",
                            hue='Nbr of countries',
                            palette='colorblind',
                            marker='.', s=100,
                            ax=ax)
        else:
            # Plot data gathered from literature review:
            sns.scatterplot(data=eu_igu["Consumption"].reset_index(),
                            x='year', y="[10^6 m²]",
                            hue='Nbr of countries',
                            palette='colorblind',
                            marker='.', s=100,
                            ax=ax)
            
        # Plot lines highlighting changes in the scope, i.e., EU enlargement:
        for y, x in eu_kdates.items():
            ax.axvline(x=x, c='grey', linestyle=':', linewidth=0.75)
            ax.text(x+0.5, 180, f"{y}", fontsize=14,
                    verticalalignment='top')
            
        ax.get_legend().remove()
        ax.set_title(f'IGU {data} [10^6 m²]', pad=30, loc='left')
        ax.yaxis.label.set_visible(False)

style_ax(ax)
ax.set_xlim(1950, 2020)
ax.set_ylim(0, 180)

plt.suptitle('European production and consumption of IGU [10^6 m²]',
             weight='light', y=1.2)

fig.subplots_adjust(wspace=0.15)

sns.despine(offset=10)
plt.show()

if export:
    # Save image:
    fig.savefig(os.path.join(path_img, 'Fig2_eu_iguProdConso.png'),
                dpi=600, bbox_inches='tight')
    fig.savefig(os.path.join(path_img, 'Fig2_eu_iguProdConso.pdf'),
                bbox_inches='tight')

# Flows of Insulating Glass Units in France and Belgium

## Creating a Dataframe for the Flows of IGU

In [ ]:
# Create a DataFrame only with production data:
fr_prod = fr_data.parse('prod').set_index('year')
be_prod = be_data.parse('prod').set_index('year')

In [ ]:
# Create a DataFrame with import and export data:
fr_imp = fr_data.parse('import').set_index('year')
fr_exp = fr_data.parse('export').set_index('year')

be_imp = be_data.parse('import').set_index('year')
be_exp = be_data.parse('export').set_index('year')

In [ ]:
# Create a DataFrame with import and export data:
fr_pop = fr_data.parse('Population').set_index('year')
be_pop = be_data.parse('Population').set_index('year')

In [ ]:
# Create a unique DataFrame for French and Belgian population:
be_fr_pop = pd.concat([fr_pop['Population, x1000'],
                       be_pop['Population, x1000']],
                      axis=1, keys=[('Population [x1000]', 'France'),
                                    ('Population [x1000]', 'Belgium')],
                      names=['Info', 'Country']).stack()

be_fr_pop = be_fr_pop.unstack()

In [ ]:
# Create a new DataFrame for Belgian and French IGU flows:
be_fr_igu = (
    pd.concat([fr_prod['IGU, kt'], be_prod['IGU, kt'],
               fr_prod['IGU, "000 m²']/1000, be_prod['IGU, "000 m²']/1000,
               fr_imp['IGU, kt'], be_imp['IGU, kt'],
               fr_imp['IGU, "000 m²']/1000, be_imp['IGU, "000 m²']/1000,
               fr_exp['IGU, kt'], be_exp['IGU, kt'],
               fr_exp['IGU, "000 m²']/1000, be_exp['IGU, "000 m²']/1000],
              axis=1,
              keys=[('Production', '[kt]', 'France'),
                    ('Production', '[kt]', 'Belgium'),
                    ('Production', '[10^6 m²]', 'France'),
                    ('Production', '[10^6 m²]', 'Belgium'),
                    ('Import', '[kt]', 'France'),
                    ('Import', '[kt]', 'Belgium'),
                    ('Import', '[10^6 m²]', 'France'),
                    ('Import', '[10^6 m²]', 'Belgium'),
                    ('Export', '[kt]', 'France'),
                    ('Export', '[kt]', 'Belgium'),
                    ('Export', '[10^6 m²]', 'France'),
                    ('Export', '[10^6 m²]', 'Belgium')],
              names=['Info', 'Unit', 'Country'])).stack(level=- 1,
                                                        dropna=False)

be_fr_igu = be_fr_igu.unstack()

In [ ]:
be_fr_igu

## Production of Insulating Glass Units

In [ ]:
countries = ['Belgium', 'France']

In [ ]:
# Plot the BE and FR production of IGU:
fig, axes = plt.subplots(nrows=1, ncols=2,
                         figsize=(16, 5),
                         sharex=True, sharey=True)

for i, (ax, country) in enumerate(zip(axes.flatten(), countries)):
    ax.plot(be_fr_igu.index,
            be_fr_igu[('Production', '[10^6 m²]')][country])

    ax.set_title(country, pad=20)
    ax.grid(which='major', axis='y', linestyle=':', linewidth=1)

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 18)

fig.suptitle('IGU production [10^6 m²/y]', y=1.1)
sns.despine(offset=5)
plt.show()

In [ ]:
# Plot a moving average for IGU production:
fig, axes = plt.subplots(nrows=1, ncols=2,
                         figsize=(16, 5),
                         sharex=True, sharey=True)

for i, (ax, country) in enumerate(zip(axes.flatten(), countries)):
    ax.plot(be_fr_igu.index,
            be_fr_igu[('Production', '[10^6 m²]')][country]
            # Linear interpolation for IGU production:
            .interpolate(method="linear", limit_area='inside')
            # Rolling IGU production according to a 5-year mean:
            .rolling(5, center=True).mean())

    ax.set_title(country, y=1.05)
    ax.set_xlim(1950, 2020)
    ax.set_ylim(0, 18)
    ax.grid(which='major', axis='y', linestyle=':', linewidth=1)

fig.suptitle('IGU production, rolled [10^6 m²/y]', fontsize=15, y=1.1)
sns.despine(offset=5)
plt.show()

## Consumption of Insulating Glass Units

In [ ]:
# Caculate import and export ratios relative to production:
be_fr_igu = be_fr_igu.stack(level=[1, 2], dropna=False)

be_fr_igu['Import Ratio'] = (be_fr_igu['Import']
                             / be_fr_igu['Production'])
be_fr_igu['Export Ratio'] = (be_fr_igu['Export']
                             / be_fr_igu['Production'])

be_fr_igu = be_fr_igu.unstack(level=[1, 2])

In [ ]:
be_fr_igu

In [ ]:
ratios = ['Import', 'Export']

In [ ]:
# Plot import and export ratios for FR and BE:
nrows = len(ratios)
ncols = len(countries)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols,
                         sharex=True, sharey=True)

for row, ratio in enumerate(ratios):
    for col, country in enumerate(countries):
        ax = axes[row][col]
        (be_fr_igu[(f"{ratio} Ratio", '[kt]', country)].plot(ax=ax))

        if row == 0:
            ax.set_title(country, y=1.1)
        if col == 0:
            ax.set_ylabel(ratio)


ax.set_ylim(0, 1)
ax.set_xlim(1950, 2020)

fig.suptitle("Import and export of IGU compared to production [%]",
             y=1.05)
sns.despine(offset=5)
plt.show()

In [ ]:
# Interpolation for missing data:
cols = [x for x in be_fr_igu.columns if x[0] in ['Import Ratio',
                                                 'Export Ratio']]

be_fr_igu[cols] = (be_fr_igu[cols].interpolate(method='linear'))

In [ ]:
be_fr_igu.loc[1990:2000, cols]

In [ ]:
# Plot import and export ratios, w/ a moving average (5-y. window length):
nrows = len(ratios)
ncols = len(countries)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols,
                         sharex=True, sharey=True)

for row, ratio in enumerate(ratios):
    for col, country in enumerate(countries):
        ax = axes[row][col]
        (be_fr_igu[(f"{ratio} Ratio", '[kt]', country)]
         .rolling(5, center=True).mean()
         .plot(ax=ax))

        ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
        if row == 0:
            ax.set_title(country, y=1.1)
        if col == 0:
            ax.set_ylabel(ratio)

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 1)

fig.suptitle("Import and export compared to production [%]")
sns.despine(offset=5)
plt.show()

In [ ]:
nrows = len(ratios)
ncols = len(countries)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols,
                         sharex=True, sharey=True)

for row, ratio in enumerate(ratios):
    for col, country in enumerate(countries):
        ax = axes[row][col]
        be_fr_igu[(f"{ratio}", '[10^6 m²]', country)].plot(ax=ax)

        ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
        if row == 0:
            ax.set_title(country, y=1.1)
        if col == 0:
            ax.set_ylabel(ratio)

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 3)

fig.suptitle('Total import and export of IGU, [10^6 m²]')
sns.despine(offset=5)
plt.show()

In [ ]:
nrows = len(ratios)
ncols = len(countries)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols,
                         sharex=True, sharey=True)

for row, ratio in enumerate(ratios):
    for col, country in enumerate(countries):
        ax = axes[row][col]
        (be_fr_igu[(f"{ratio}", "[10^6 m²]", country)]
         .interpolate(method='linear', limit_area='inside')
         .rolling(5, center=True).mean()
         .plot(ax=ax))

        ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
        if row == 0:
            ax.set_title(country, y=1.1)
        if col == 0:
            ax.set_ylabel(ratio)

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 3)

fig.suptitle('Total import and export of IGU, 5-y rolled, [10^6 m²]')
sns.despine(offset=5)
plt.show()

In [ ]:
be_fr_igu = be_fr_igu.stack(level=[1, 2], dropna=False)

be_fr_igu['Consumption'] = (be_fr_igu['Production']
                            + be_fr_igu['Import']
                            - be_fr_igu['Export']
                            )

be_fr_igu = be_fr_igu.unstack(level=[1, 2])

In [ ]:
# Plot total consumption of IGU:
fig, axes = plt.subplots(nrows=1, ncols=2,
                         figsize=(16, 5),
                         sharex=True, sharey=True)

for i, (ax, country) in enumerate(zip(axes.flatten(), countries)):

    ax.plot(be_fr_igu.index,
            be_fr_igu['Consumption', '[10^6 m²]'][country])
    ax.set_title(country)

ax.set_xlim(1940, 2020)
ax.set_ylim(ymin=0)

fig.suptitle('IGU Consumption, total [10^6 m²/y]', fontsize=15, y=1.05)
sns.despine(offset=5)
plt.show()

In [ ]:
# Linear interpolation for IGU consumption
# and moving average with a 5-year window length:
fig, axes = plt.subplots(nrows=1, ncols=2,
                         figsize=(16, 5),
                         sharex=True, sharey=True)

for i, (ax, country) in enumerate(zip(axes.flatten(), countries)):
    ax.plot(be_fr_igu.index,
            be_fr_igu['Consumption', '[10^6 m²]'][country]
            .interpolate(method="linear", limit_area='inside')
            .rolling(5, center=True).mean())
    ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
    ax.set_title(country)

ax.set_xlim(1945, 2020)
ax.set_ylim(0, 20)

fig.suptitle('IGU Consumption, rolled [10^6 m²/y]',
             fontsize=15, y=1.05)
sns.despine(offset=5)
plt.show()

In [ ]:
# Estimating the IGU consumption per capita
for country in be_fr_igu['Consumption', '[10^6 m²]'].columns:
    be_fr_igu[('Consumption', '[m²/cap]', country)] = (
        be_fr_igu[('Consumption', '[10^6 m²]', country)]*1000
        / be_fr_pop[('Population [x1000]', country)])

In [ ]:
# Linear interpolation for IGU consumption per capita
# and moving average with a 5-y. window length:
fig, axes = plt.subplots(nrows=1, ncols=2,
                         figsize=(16, 5),
                         sharex=True, sharey=True)

for i, (ax, country) in enumerate(zip(axes.flatten(), countries)):
    ax.plot(be_fr_igu.index,
            be_fr_igu['Consumption', '[m²/cap]'][country]
            .interpolate(method="linear", limit_area='inside')
            .rolling(5, center=True).mean())

    ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
    ax.set_title(country, y=1.1)
    ax.set_xlim(1945, 2020)
    ax.set_ylim(0, 0.5)

fig.suptitle('IGU consumption, m² per capita, rolled',
             fontsize=15, y=1.05)
sns.despine(offset=5)
plt.show()

In [ ]:
be_fr_igu = be_fr_igu.drop('[m²/cap]', axis=1, level=1)

## Summary

In [ ]:
# Plot total import and export flows, moving average w/ a 5-y. window length:
toplot = ['Production', 'Export', 'Import', 'Consumption']
nrows = len(toplot)
ncols = len(countries)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols,
                         sharex=True, sharey=True,
                         figsize=(13, 14))

for row, data in enumerate(toplot):
    for col, country in enumerate(countries):
        ax = axes[row][col]
        # Define the color and title for the subplots:
        if country == 'Belgium':
            color = 'forestgreen'
            ax.set_title(f"{data} [10^6 m²]", loc='left', pad=15)
        else:
            color = 'steelblue'

        (be_fr_igu[(f"{data}", "[10^6 m²]", country)]
         .interpolate(method='linear')
         .rolling(5, center=True).mean()
         .plot(ax=ax, c=color)
         )

        x = be_fr_igu.index
        y = (be_fr_igu[(f"{data}", "[10^6 m²]", country)]
             .interpolate(method='linear')
             .rolling(5, center=True).mean())
        ax.fill_between(x, y, color=color, alpha=0.075)

        sns.scatterplot(data=be_fr_igu[f"{data}", "[10^6 m²]"][country],
                        marker='.', s=50, color='grey',
                        ax=ax)

        ax.get_legend().remove()

        ax.minorticks_off()
        ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
        style_ax(ax)
        ax.set_ylabel(None)

        ax.grid(which='major', axis='y', linestyle=':', linewidth=1)

        if row == 0:
            ax.set_title(country, y=1.25)

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 20)
plt.xticks(np.arange(1950, 2021, 10))

fig.subplots_adjust(wspace=0.15, hspace=0.5)

fig.suptitle("Flows of IGU [10^6 m²]", y=1)
sns.despine(offset=5)

if export:
    # Save image:
    fig.savefig(os.path.join(path_img, 'AppendixC_BEFR_IGUFlows.png'),
                dpi=600, bbox_inches='tight')
    fig.savefig(os.path.join(path_img, 'AppendixC_BEFR_IGUFlows.pdf'),
                bbox_inches='tight')

plt.show()

Hypothesis for missing data: French production = French consumption

In [ ]:
be_fr_igu = be_fr_igu.stack(level=[1, 2], dropna=False)

be_fr_igu["Estimated consumption"] = (
    be_fr_igu["Consumption"]
    .fillna(be_fr_igu["Production"])
)

be_fr_igu = be_fr_igu.unstack(level=[1, 2])

In [ ]:
# Plot total import and export flows, moving average w/ a 5-y. window length:
toplot = ['Production', 'Export', 'Import', 'Consumption']
nrows = len(toplot)
ncols = len(countries)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols,
                         sharex=True, sharey=True,
                         figsize=(13, 14))

for row, data in enumerate(toplot):
    for col, country in enumerate(countries):
        ax = axes[row][col]
        # Define the color and title for the subplots:
        if country == 'Belgium':
            color = 'forestgreen'
            ax.set_title(f"{data} [10^6 m²]", loc='left', pad=15)
        else:
            color = 'steelblue'

        if row == 3:
            sns.lineplot(data=be_fr_igu[("Estimated consumption", "[10^6 m²]",
                                         country)]
                         .interpolate(method='linear')
                         .rolling(5, center=True).mean(),
                         linestyle='--', color='darkgrey', ax=ax)

        (be_fr_igu[(f"{data}", "[10^6 m²]", country)]
         .interpolate(method='linear')
         .rolling(5, center=True).mean()
         .plot(ax=ax, c=color)
         )

        x = be_fr_igu.index
        y = (be_fr_igu[(f"{data}", "[10^6 m²]", country)]
             .interpolate(method='linear')
             .rolling(5, center=True).mean())
        ax.fill_between(x, y, color=color, alpha=0.075)

        sns.scatterplot(data=be_fr_igu[f"{data}", "[10^6 m²]"][country],
                        marker='.', s=50, color='grey',
                        ax=ax)

        ax.get_legend().remove()

        ax.minorticks_off()
        ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
        style_ax(ax)
        ax.set_ylabel(None)

        ax.grid(which='major', axis='y', linestyle=':', linewidth=1)

        if row == 0:
            ax.set_title(country, y=1.25)

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 20)
plt.xticks(np.arange(1950, 2021, 10))

fig.subplots_adjust(wspace=0.15, hspace=0.5)

fig.suptitle("Flows of IGU [10^6 m²]", y=1)
sns.despine(offset=5)

if export:
    # Save image:
    fig.savefig(os.path.join(path_img, 'AppendixC_BEFR_IGUFlows.png'),
                dpi=600, bbox_inches='tight')
    fig.savefig(os.path.join(path_img, 'AppendixC_BEFR_IGUFlows.pdf'),
                bbox_inches='tight')

plt.show()

# The Environmental Footprint Relating to the Production (and consumption) of IGU in the EU

## Raw Material Use for IGU Production

In [ ]:
# Absolute use of raw materials for IGU production:
for material in materials:
    eu_igu[(f"{material}, IGU production", '[kt]')] = (
        eu_igu[('Production', '[kt]')]
        * (mat_intensity[(f"{material}, kg/kg")]
           .interpolate(method='pchip', limit_direction='forward'))
    )

In [ ]:
# Plot raw material flows for IGU production in the EU:
fig, axes = plt.subplots(nrows=3, ncols=2,
                         sharex=True, sharey=True,
                         figsize=(10, 8))

for i, (ax, material) in enumerate(zip(axes.flatten(), materials)):
    sns.lineplot(data=eu_igu[f"{material}, IGU production"].reset_index(),
                 x='year', y='[kt]',
                 hue='Nbr of countries',
                 palette='colorblind',
                 ax=ax)
    ax.set_title(material)
    ax.set_ylabel(None)
    style_ax(ax)
    ax.get_legend().remove()

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 2000)

fig.suptitle("Raw material flows related to IGU production in the EU [kt/y]")
fig.subplots_adjust(hspace=0.4)
sns.despine(offset=5)
plt.show()

In [ ]:
# Plot raw material flows for EU IGU production
fig, axes = plt.subplots(nrows=3, ncols=2,
                         sharex=True, sharey=True,
                         figsize=(10, 8))

for i, (ax, material) in enumerate(zip(axes.flatten(), materials)):
    for a, b in eu_list:
        ax.plot(eu_igu.interpolate()
                .rolling(5, center=True).mean().loc[a:b]
                .reset_index(level='Nbr of countries')
                .sort_values(by=['year'])
                [(f"{material}, IGU production", '[kt]')],
                linewidth=1.1, c='black', linestyle=':'
                )
        ax.set_title(material)
        style_ax(ax)

ax.set_xlim(1950, 2020)
plt.xticks(np.arange(1950, 2021, 10))
ax.set_ylim(0, 2000)

fig.suptitle("Raw material flows relating to IGU production in the EU [kt/y]")
fig.subplots_adjust(hspace=0.4)
sns.despine(offset=5)
plt.show()

In [ ]:
# Estimate total use of raw materials (sum per material):
eu_igu[('Total raw materials, IGU production', '[Mt]')] = 0

for material in materials:
    eu_igu[('Total raw materials, IGU production', '[Mt]')] = (
        eu_igu[(f"{material}, IGU production", '[kt]')]/1000
        + eu_igu[('Total raw materials, IGU production', '[Mt]')]
    )

In [ ]:
# Plot total material use for IGU production in the EU:
fig, ax = plt.subplots(figsize=(8, 5))

sns.lineplot(data=eu_igu['Total raw materials, IGU production'].reset_index(),
             x='year', y='[Mt]',
             hue='Nbr of countries',
             palette='colorblind',
             ax=ax)

ax.set_xlim(1950, 2020)
ax.set_ylim(0,4)
ax.set_ylabel(None)
style_ax(ax)
ax.get_legend().remove()

fig.suptitle("Total use of raw materials for IGU production in the EU [Mt/y]")
fig.subplots_adjust(hspace=0.4)
sns.despine(offset=5)
plt.show()

In [ ]:
# Plot total material use for IGU production in the EU:
fig, ax = plt.subplots(figsize=(8, 5))

for a, b in eu_list:
    ax.plot(eu_igu.interpolate().rolling(5, center=True).mean()
            .loc[a:b]
            .reset_index(level='Nbr of countries')
            .sort_values(by=['year'])
            [('Total raw materials, IGU production', '[Mt]')],
            linewidth=1.1, c='black', linestyle=':'
            )

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 4)
style_ax(ax)

fig.suptitle("Total use of raw materials for IGU production in the EU [Mt]")
fig.subplots_adjust(hspace=0.4)
sns.despine(offset=5)
plt.show()

## Energy use for the production of IGU

In [ ]:
# Absolute use of energy for IGU production:
for energy in energies:
    eu_igu[(f"{energy}, IGU production", '[PJ]')] = (
        eu_igu[('Production', '[kt]')]
        * (energy_intensity[f"{energy}, GJ/t"]
           .interpolate(method='pchip', limit_direction='forward'))/1000
    )

In [ ]:
# Plot energy used for IGU production in EU:
fig, axes = plt.subplots(nrows=1, ncols=3,
                         sharex=True, sharey=True,
                         figsize=(16, 4))

for i, (ax, energy) in enumerate(zip(axes.flatten(), energies)):
    sns.lineplot(data=eu_igu[f"{energy}, IGU production"].reset_index(),
                 x='year', y="[PJ]",
                 hue='Nbr of countries',
                 palette='colorblind',
                 ax=ax)
    ax.set_title(energy)
    ax.get_legend().remove()
    ax.set_ylabel(None)
    style_ax(ax)

ax.set_xlim(1950, 2020)
plt.xticks(np.arange(1950, 2021, 10))
ax.set_ylim(0, 25)

fig.suptitle("Energy used in the production of IGU in the EU [PJ]", y=1.1)
fig.subplots_adjust(hspace=0.4)
sns.despine(offset=5)
plt.show()

In [ ]:
# Plot energy flows for IGU production, 5-y rolled:
fig, axes = plt.subplots(nrows=1, ncols=3,
                         sharex=True, sharey=True,
                         figsize=(16, 4))

for i, (ax, energy) in enumerate(zip(axes.flatten(), energies)):
    for a, b in eu_list:
        ax.plot(eu_igu.interpolate()
                .rolling(5, center=True).mean().loc[a:b]
                .reset_index(level='Nbr of countries')
                .sort_values(by=['year'])
                [(f"{energy}, IGU production", '[PJ]')],
                linewidth=1.1, c='black', linestyle=':'
                )
        ax.set_title(energy)
        style_ax(ax)

ax.set_xlim(1950, 2020)
plt.xticks(np.arange(1950, 2021, 10))
ax.set_ylim(0, 25)

fig.suptitle("Raw material flows relating to IGU production in the EU [kt/y]", 
             y=1.1)
fig.subplots_adjust(hspace=0.4)
sns.despine(offset=5)
plt.show()

In [ ]:
eu_igu[('Total energy, IGU production', '[PJ]')] = 0

for energy in energies:
    eu_igu[('Total energy, IGU production', '[PJ]')] = (
        eu_igu[(f"{energy}, IGU production", '[PJ]')]
        + eu_igu[('Total energy, IGU production', '[PJ]')]
    )

In [ ]:
# Plot total energy use for IGU production in the EU:
fig, ax = plt.subplots(figsize=(8, 5))

sns.lineplot(data=eu_igu['Total energy, IGU production'].reset_index(),
             x='year', y='[PJ]',
             hue='Nbr of countries',
             palette='colorblind',
             ax=ax)

ax.set_xlim(1950, 2020)
plt.xticks(np.arange(1950, 2021, 10))
ax.set_ylim(0, 40)
ax.get_legend().remove()
ax.set_ylabel(None)
style_ax(ax)

fig.suptitle("Total use of energy for IGU production in the EU [PJ]")
fig.subplots_adjust(hspace=0.4)
sns.despine(offset=5)
plt.show()

In [ ]:
# Plot total energy use for IGU production in the EU:
fig, ax = plt.subplots(figsize=(8, 5))

for a, b in eu_list:
    ax.plot(eu_igu.interpolate().rolling(5, center=True).mean()
            .loc[a:b]
            .reset_index(level='Nbr of countries')
            .sort_values(by=['year'])
            [('Total energy, IGU production', '[PJ]')],
            linewidth=1.1, c='black', linestyle=':'
            )

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 40)
style_ax(ax)

fig.suptitle("Total use of energy for IGU production in the EU [PJ]")
fig.subplots_adjust(hspace=0.4)
sns.despine(offset=5)
plt.show()

## Absolute CO2 emissions relating to the production of IGU, EU

In [ ]:
# Estimating the co2 emission related to IGU production
eu_igu[('co2, IGU production', '[kt]')] = (
    eu_igu[('Production', '[kt]')]
    * co2_intensity['co2 interpolated [kg/t]'] / 1000
)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

sns.lineplot(data=eu_igu['co2, IGU production'].reset_index(),
             x='year', y='[kt]',
             hue='Nbr of countries',
             palette='colorblind',
             ax=ax)

ax.set_xlim(1980, 2020)
ax.set_ylim(0, 2000)
ax.get_legend().remove()

fig.suptitle("co2 emissions for EU IGU production [kt]")
fig.subplots_adjust(hspace=0.4)
sns.despine(offset=5)
plt.show()

In [ ]:
# Plot total energy use for IGU production in the EU:
fig, ax = plt.subplots(figsize=(8, 5))

for a, b in eu_list:
    ax.plot(eu_igu.rolling(5, center=True).mean().loc[a:b]
            .reset_index(level='Nbr of countries')
            .sort_values(by=['year'])
            [('co2, IGU production', '[kt]')],
            linewidth=1.1, c='black', linestyle=':'
            )

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 1800)

fig.suptitle("CO2 emissions for EU IGU production [kt]")
fig.subplots_adjust(hspace=0.4)
sns.despine(offset=5, trim=True)
plt.show()

## Summary

In [ ]:
# Number of countries for trend curves:
c = [(6, 12), (10, 15), (27, 28)]

In [ ]:
# Plot the absolute use of raw materials for IGU prod in the EU:
fig, axes = plt.subplots(nrows=2, ncols=3,
                         sharex=True, sharey=True,
                         figsize=(16, 5))

for i, (ax, material) in enumerate(zip(axes.flatten(), materials)):
    for a, b in c:
        ax.plot(eu_igu.interpolate().rolling(5, center=True).mean().loc[a:b]
                .reset_index(level='Nbr of countries')
                .sort_values(by=['year'])
                [f"{material}, IGU production", '[kt]'],
                linewidth=1.1, c='sandybrown', linestyle='-')

    x = (eu_igu.reset_index()['year'])
    y = (eu_igu[(f"{material}, IGU production", '[kt]')].interpolate()
         .rolling(5, center=True).mean())

    ax.fill_between(x, y, color="sandybrown", alpha=0.1)
    ax.grid(which='major', axis='y', linestyle=':', linewidth=1)

    ax.set_title(f" {material}", loc='left')
    style_ax(ax)

ax.set_ylim(0, 2000)
ax.set_xlim(1950, 2020)
plt.yticks(np.arange(0, 2001, 500))
plt.xticks(np.arange(1950, 2021, 10))

# Divide by 1000 the y-axis, results in Mt:
ax.yaxis.set_major_formatter(y_1000)

fig.suptitle("(a) Use of raw materials [Mt]", x=0.1, y=1, ha='left')
fig.subplots_adjust(wspace=0.15, hspace=0.4)

sns.despine(offset=5)

if export:
    # Save image:
    fig.savefig(os.path.join(path_img, 'Fig5_EU_Use of raw materials.png'),
                dpi=600, bbox_inches='tight')
    fig.savefig(os.path.join(path_img, 'Fig5_EU_Use of raw materials.pdf'),
                bbox_inches='tight')

plt.show()

In [ ]:
# Plot the absolute use of energy in the EU for architectural glass prod:
fig, axes = plt.subplots(nrows=1, ncols=3,
                         sharex=True, sharey=True,
                         figsize=(16, 2.5))

for i, (ax, energy) in enumerate(zip(axes.flatten(), energies)):
    for a, b in c:
        ax.plot(eu_igu.interpolate().rolling(5, center=True).mean().loc[a:b]
                .reset_index(level='Nbr of countries')
                .sort_values(by=['year'])
                [(f"{energy}, IGU production", '[PJ]')],
                linewidth=1.1, c='steelblue', linestyle='-')

    x = (eu_igu.reset_index()['year'])
    y = (eu_igu[(f"{energy}, IGU production", '[PJ]')].interpolate()
         .rolling(5, center=True).mean())
    ax.fill_between(x, y, color="steelblue", alpha=0.10)
    ax.grid(which='major', axis='y', linestyle=':', linewidth=1)

    ax.set_title(f" {energy}", loc='left')
    style_ax(ax)

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 25)
plt.xticks(np.arange(1950, 2021, 10))

fig.suptitle("(b) Use of energy [PJ]", x=0.1, y=1.12, ha='left')
fig.subplots_adjust(wspace=0.15)

sns.despine(offset=5)

if export:
    # Save image:
    fig.savefig(os.path.join(path_img, 'Fig5_EU_Use of energy.png'),
                dpi=600, bbox_inches='tight')
    fig.savefig(os.path.join(path_img, 'Fig5_EU_Use of energy.pdf'),
                bbox_inches='tight')

plt.show()

In [ ]:
# Plot total energy and material use, and total CO2 emissions for IGU prod:
fig, axes = plt.subplots(nrows=1, ncols=3,
                         sharex=True,
                         figsize=(16, 2.5))

# Total use of raw materials:
ax = axes[0]
for a, b in c:
    ax.plot(eu_igu.interpolate().rolling(5, center=True).mean().loc[a:b]
            .reset_index(level='Nbr of countries')
            .sort_values(by=['year'])
            [('Total raw materials, IGU production', '[Mt]')],
            linewidth=1.1, c='firebrick', linestyle='-')

x = (eu_igu.reset_index()['year'])
y = (eu_igu[('Total raw materials, IGU production', '[Mt]')].interpolate()
     .rolling(5, center=True).mean())
ax.fill_between(x, y, color="firebrick", alpha=0.1)

ax.grid(which='major', axis='y', linestyle=':', linewidth=1)

ax.set_title("Raw material use [Mt]", loc='left')
style_ax(ax)
ax.set_ylim(0, 4)
ax.set_xlim(1950, 2020)
plt.xticks(np.arange(1950, 2021, 10))

# Total use of energy:
ax = axes[1]
for a, b in c:
    ax.plot(eu_igu.interpolate().rolling(5, center=True).mean().loc[a:b]
            .reset_index(level='Nbr of countries')
            .sort_values(by=['year'])
            [('Total energy, IGU production', '[PJ]')],
            linewidth=1.1, c='firebrick', linestyle='-')

x = (eu_igu.reset_index()['year'])
y = (eu_igu[('Total energy, IGU production', '[PJ]')].interpolate()
     .rolling(5, center=True).mean())
ax.fill_between(x, y, color="firebrick", alpha=0.1)

ax.grid(which='major', axis='y', linestyle=':', linewidth=1)

ax.set_title("Energy use [PJ]", loc='left')
style_ax(ax)
ax.set_ylim(0, 40)

# Total emissions of CO2:
ax = axes[2]
for a, b in c:
    ax.plot(eu_igu.interpolate().rolling(5, center=True).mean().loc[a:b]
            .reset_index(level='Nbr of countries')
            .sort_values(by=['year'])
            [('co2, IGU production', '[kt]')],
            linewidth=1.1, c='firebrick', linestyle='-')

x = (eu_igu.reset_index()['year'])
y = (eu_igu[('co2, IGU production', '[kt]')].interpolate()
     .rolling(5, center=True).mean())
ax.fill_between(x, y, color="firebrick", alpha=0.1)

ax.grid(which='major', axis='y', linestyle=':', linewidth=1)

# Divide by 1000 the y-axis, results in Mt
ax.yaxis.set_major_formatter(y_1000)

ax.set_title("CO2 emissions [MtCO2]", loc='left')
style_ax(ax)
ax.set_ylim(0, 2000)

fig.suptitle("(c) Total flows of raw materials, energy and CO2",
             x=0.1, y=1.12, ha='left')
fig.subplots_adjust(wspace=0.15)

sns.despine(offset=5)

if export:
    # Save image:
    plt.savefig(os.path.join(path_img, 'Fig5_EU_Total flows.png'),
                dpi=600, bbox_inches='tight')
    plt.savefig(os.path.join(path_img, 'Fig5_EU_Total flows.pdf'),
                bbox_inches='tight')

plt.show()

# The Environmental Footprint of IGU used in BE and FR

## Raw Material Use for IGU Production

In [ ]:
# Absolute use of raw materials for IGU consumption:
for mat in materials:
    for country in be_fr_igu[('Estimated consumption', '[kt]')].columns:
        be_fr_igu[(f"{mat}, IGU consum", '[kt]', country)] = (
            be_fr_igu[('Estimated consumption', '[kt]', country)]
            * (mat_intensity[(f"{mat}, kg/kg")]
               .interpolate(method='pchip', limit_direction='forward'))
        )

In [ ]:
nrows = len(materials)
ncols = len(countries)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols,
                         sharex=True, sharey=True,
                         figsize=(8, 16))

for row, mat in enumerate(materials):
    for col, country in enumerate(countries):
        ax = axes[row][col]
        if row == 0:
            ax.set_title(country)
        if col == 0:
            ax.set_ylabel(mat)
        be_fr_igu[(f"{mat}, IGU consum", '[kt]', country)].plot(ax=ax)

ax.set_xlim(1950, 2020)

sns.despine()
plt.show()

In [ ]:
for country in countries:
    be_fr_igu[('Total raw material use, IGU consum', '[kt]', country)] = 0

for mat in materials:
    be_fr_igu['Total raw material use, IGU consum', '[kt]'] = (
        be_fr_igu[f"{mat}, IGU consum", '[kt]']
        + be_fr_igu[('Total raw material use, IGU consum', '[kt]')]
    )

In [ ]:
ncols = len(countries)

fig, axes = plt.subplots(nrows=1, ncols=ncols,
                         sharex=True, sharey=True,
                         figsize=(16, 5))

for col, country in enumerate(countries):
    ax = axes[col]
    (be_fr_igu[('Total raw material use, IGU consum', '[kt]', country)]
     .plot(ax=ax))

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 500)
sns.despine()
plt.show()

In [ ]:
ncols = len(countries)

fig, axes = plt.subplots(nrows=1, ncols=ncols,
                         sharex=True, sharey=True,
                         figsize=(16, 5))

for col, country in enumerate(countries):
    ax = axes[col]
    (be_fr_igu[('Total raw material use, IGU consum', '[kt]', country)]
     .rolling(5, center=True).mean()
     .plot(ax=ax))

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 500)
sns.despine()
plt.show()

## Energy use for IGU Production

In [ ]:
# Absolute use of energy for IGU production:
for energy in energies:
    for country in countries:
        be_fr_igu[(f"{energy}, IGU consum", '[TJ]', country)] = (
            be_fr_igu[('Estimated consumption', '[kt]', country)]
            * (energy_intensity[(f"{energy}, GJ/t")]
               .interpolate(method='pchip', limit_direction='forward'))
        )

In [ ]:
nrows = len(energies)
ncols = len(countries)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols,
                         sharex=True, sharey=True,
                         figsize=(16, 10))

for row, energy in enumerate(energies):
    for col, country in enumerate(countries):
        ax = axes[row][col]
        if row == 0:
            ax.set_title(country)
        if col == 0:
            ax.set_ylabel(energy)
        be_fr_igu[(f"{energy}, IGU consum", '[TJ]', country)].plot(ax=ax)

ax.set_xlim(1950, 2020)
ax.set_ylim(ymin=0)

sns.despine()
plt.show()

In [ ]:
# Total energy use for IGU production (sum of energy sources):
for country in countries:
    be_fr_igu[('Total energy use, IGU consum', '[TJ]', country)] = 0

for energy in energies:
    be_fr_igu[('Total energy use, IGU consum', '[TJ]')] = (
        be_fr_igu[f"{energy}, IGU consum", '[TJ]']
        + be_fr_igu[('Total energy use, IGU consum', '[TJ]')]
    )

In [ ]:
ncols = len(countries)

fig, axes = plt.subplots(nrows=1, ncols=ncols,
                         sharex=True, sharey=True,
                         figsize=(16, 5))

for col, country in enumerate(countries):
    ax = axes[col]
    (be_fr_igu[('Total energy use, IGU consum', '[TJ]', country)]
     .plot(ax=ax))

ax.set_xlim(1945, 2020)
ax.set_ylim(0, 5000)
sns.despine()
plt.show()

In [ ]:
ncols = len(countries)

fig, axes = plt.subplots(nrows=1, ncols=ncols,
                         sharex=True, sharey=True,
                         figsize=(16, 5))

for col, country in enumerate(countries):
    ax = axes[col]
    (be_fr_igu[('Total energy use, IGU consum', '[TJ]', country)]
     .rolling(5, center=True).mean()
     .plot(ax=ax))

ax.set_xlim(1945, 2020)
ax.set_ylim(0, 5000)
sns.despine()
plt.show()

## CO2 emissions Relating to IGU Consumption in France and Belgium

In [ ]:
# Estimating the CO2 emission related to IGU consumption
for country in be_fr_igu[('Estimated consumption', '[kt]')].columns:
    be_fr_igu[('CO2, IGU consum', '[ktCO2]', country)] = (
        be_fr_igu[('Estimated consumption', '[kt]', country)]
        * co2_intensity['co2 interpolated [kg/t]'] / 1000
    )

In [ ]:
# Estimating the CO2 emission related to IGU production
for country in be_fr_igu[('Production', '[kt]')].columns:
    be_fr_igu[('CO2, IGU production', '[ktCO2]', country)] = (
        be_fr_igu[('Production', '[kt]', country)]
        * co2_intensity['co2 interpolated [kg/t]'] / 1000
    )

In [ ]:
co2_igu = ['production', 'consum']

In [ ]:
nrows = len(co2_igu)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols,
                         sharex=True, sharey=True,
                         figsize=(8, 8))

for row, co2 in enumerate(co2_igu):
    for col, country in enumerate(countries):
        ax = axes[row][col]
        if row == 0:
            ax.set_title(country)
        if col == 0:
            ax.set_ylabel(co2)
        be_fr_igu[(f"CO2, IGU {co2}", '[ktCO2]', country)].plot(ax=ax)

ax.set_xlim(1950, 2020)

sns.despine()
plt.show()

In [ ]:
nrows = len(co2_igu)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols,
                         sharex=True, sharey=True,
                         figsize=(8, 8))

for row, co2 in enumerate(co2_igu):
    for col, country in enumerate(countries):
        ax = axes[row][col]
        if row == 0:
            ax.set_title(country)
        if col == 0:
            ax.set_ylabel(co2)
        (be_fr_igu[(f"CO2, IGU {co2}", '[ktCO2]', country)]
         .rolling(5, center=True).mean()
         .plot(ax=ax))

ax.set_xlim(1950, 2020)

sns.despine()
plt.show()

## Summary

In [ ]:
# Raw materials, per type:
mat_reduced = ['sand', 'cullet', 'sodium carbonate']

nrows = len(mat_reduced)
ncols = len(countries)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols,
                         sharex=True, sharey=True,
                         figsize=(10, 8))

for row, material in enumerate(mat_reduced):
    for col, country in enumerate(countries):
        ax = axes[row][col]
        if country == 'Belgium':
            color = 'sandybrown'
            ax.set_title(f" {material}", loc='left')
        else:
            color = 'saddlebrown'

        (be_fr_igu[(f"{material}, IGU consum", '[kt]', country)]
         .interpolate(method="linear", limit_area='inside')
         .rolling(5, center=True).mean()
         .plot(ax=ax, c=color))

        x = be_fr_igu.index
        y = (be_fr_igu[(f"{material}, IGU consum", '[kt]', country)]
             .interpolate(method="linear", limit_area='inside')
             .rolling(5, center=True).mean())
        ax.fill_between(x, y, color="sandybrown", alpha=0.1)

        ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
        style_ax(ax)
        if row == 0:
            ax.set_title(country, y=1.2)

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 250)
plt.xticks(np.arange(1950, 2021, 10))
plt.yticks(np.arange(0, 251, 50))

fig.suptitle("(a) Use of raw materials, per type [kt]",
             x=0.07, y=1, ha='left')
fig.subplots_adjust(wspace=0.15, hspace=0.4)

sns.despine(offset=5)
plt.show()

if export:
    # Save image:
    fig.savefig(os.path.join(path_img, 'Fig6_BEFR_Use of raw materials.png'),
                dpi=600, bbox_inches='tight')
    fig.savefig(os.path.join(path_img, 'Fig6_BEFR_Use of raw materials.pdf'),
                bbox_inches='tight')

In [ ]:
# Raw materials, total:
ncols = len(countries)

fig, axes = plt.subplots(nrows=1, ncols=ncols,
                         sharex=True, sharey=True,
                         figsize=(10, 2.1))

for col, country in enumerate(countries):
    ax = axes[col]
    if country == 'Belgium':
        color = 'firebrick'
    else:
        color = 'maroon'

    (be_fr_igu[("Total raw material use, IGU consum", '[kt]', country)]
     .interpolate(method="linear", limit_area='inside')
     .rolling(5, center=True).mean()
     .plot(ax=ax, c=color))

    x = be_fr_igu.index
    y = (be_fr_igu[("Total raw material use, IGU consum", '[kt]', country)]
         .interpolate(method="linear", limit_area='inside')
         .rolling(5, center=True).mean())
    ax.fill_between(x, y, color="firebrick", alpha=0.1)

    ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
    style_ax(ax)
    ax.set_title(country, y=1.05)

ax.set_xlim(1950, 2020)
plt.xticks(np.arange(1950, 2021, 10))

ax.set_ylim(0, 500)
plt.yticks(np.arange(0, 501, 100))

fig.suptitle("(b) Use of raw materials, total [kt]",
             x=0.07, y=1.2, ha='left')
fig.subplots_adjust(wspace=0.15, hspace=0.4)

sns.despine(offset=5)
plt.show()

if export:
    # Save image:
    fig.savefig(os.path.join(path_img, 'Fig6_BEFR_Total raw materials.png'),
                dpi=600, bbox_inches='tight')
    fig.savefig(os.path.join(path_img, 'Fig6_BEFR_Total raw materials.pdf'),
                bbox_inches='tight')

In [ ]:
# Energy use, per type:
nrows = len(energies)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols,
                         sharex=True, sharey=True,
                         figsize=(10, 8))

for row, energy in enumerate(energies):
    for col, country in enumerate(countries):
        ax = axes[row][col]
        if country == 'Belgium':
            color = 'cornflowerblue'
            ax.set_title(f" {energy}", loc='left')
        else:
            color = 'darkblue'

        (be_fr_igu[(f"{energy}, IGU consum", '[TJ]', country)]
         .interpolate(method="linear", limit_area='inside')
         .rolling(5, center=True).mean()
         .plot(ax=ax, c=color)
         )

        x = be_fr_igu.index
        y = (be_fr_igu[(f"{energy}, IGU consum", '[TJ]', country)]
             .interpolate(method="linear", limit_area='inside')
             .rolling(5, center=True).mean())
        ax.fill_between(x, y, color="steelblue", alpha=0.1)

        # Divide by 1000 the y-axis, results in PJ
        ax.yaxis.set_major_formatter(y_1000)

        ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
        style_ax(ax)
        if row == 0:
            ax.set_title(country, y=1.2)

ax.set_xlim(1950, 2020)
ax.set_ylim(0, 3000)

plt.xticks(np.arange(1950, 2021, 10))

fig.suptitle("(a) Use of energy, per type [PJ]",
             x=0.07, y=1, ha='left')
fig.subplots_adjust(wspace=0.15, hspace=0.4)

sns.despine(offset=5)
plt.show()

if export:
    # Save image:
    fig.savefig(os.path.join(path_img, 'Fig7_BEFR_Use of energy.png'),
                dpi=600, bbox_inches='tight')
    fig.savefig(os.path.join(path_img, 'Fig7_BEFR_Use of energy.pdf'),
                bbox_inches='tight')

In [ ]:
# Energy use, total:
fig, axes = plt.subplots(nrows=1, ncols=ncols,
                         sharex=True, sharey=True,
                         figsize=(10, 2.1))

for col, country in enumerate(countries):
    ax = axes[col]
    ax = axes[col]
    if country == 'Belgium':
        color = 'firebrick'
    else:
        color = 'maroon'

    (be_fr_igu[('Total energy use, IGU consum', '[TJ]', country)]
     .interpolate(method="linear", limit_area='inside')
     .rolling(5, center=True).mean()
     .plot(ax=ax, c=color))

    x = be_fr_igu.index
    y = (be_fr_igu[('Total energy use, IGU consum', '[TJ]', country)]
         .interpolate(method="linear", limit_area='inside')
         .rolling(5, center=True).mean())
    ax.fill_between(x, y, color="firebrick", alpha=0.1)

    # Divide by 1000 the y-axis, results in PJ
    ax.yaxis.set_major_formatter(y_1000)

    ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
    style_ax(ax)
    ax.set_title(country, y=1.05)

ax.set_xlim(1950, 2020)
plt.xticks(np.arange(1950, 2021, 10))

ax.set_ylim(0, 5000)
plt.yticks(np.arange(0, 5001, 1000))

fig.suptitle("(b) Use of energy, total [PJ]",
             x=0.07, y=1.2, ha='left')
fig.subplots_adjust(wspace=0.15, hspace=0.4)

sns.despine(offset=5)
plt.show()

if export:
    # Save image:
    fig.savefig(os.path.join(path_img, 'Fig7_BEFR_Total enrgy.png'),
                dpi=600, bbox_inches='tight')
    fig.savefig(os.path.join(path_img, 'Fig7_BEFR_Total enrgy.pdf'),
                bbox_inches='tight')

In [ ]:
# CO2 emissions, total:
fig, axes = plt.subplots(nrows=1, ncols=ncols,
                         sharex=True, sharey=True,
                         figsize=(10, 2.1))

for col, country in enumerate(countries):
    ax = axes[col]
    ax = axes[col]
    if country == 'Belgium':
        color = 'firebrick'
    else:
        color = 'maroon'

    (be_fr_igu[('CO2, IGU consum', '[ktCO2]', country)]
     .interpolate(method="linear", limit_area='inside')
     .rolling(5, center=True).mean()
     .plot(ax=ax, c=color))

    x = be_fr_igu.index
    y = (be_fr_igu[('CO2, IGU consum', '[ktCO2]', country)]
         .interpolate(method="linear", limit_area='inside')
         .rolling(5, center=True).mean())
    ax.fill_between(x, y, color="firebrick", alpha=0.1)

    ax.grid(which='major', axis='y', linestyle=':', linewidth=1)
    style_ax(ax)
    ax.set_title(country, y=1.05)

ax.set_xlim(1950, 2020)
plt.xticks(np.arange(1950, 2021, 10))
ax.set_ylim(0, 250)
plt.yticks(np.arange(0, 251, 50))

fig.suptitle("(c) CO2 emissions, total [ktCO2]",
             x=0.07, y=1.2, ha='left')
fig.subplots_adjust(wspace=0.15, hspace=0.4)

sns.despine(offset=5)
plt.show()

if export:
    # Save image:
    fig.savefig(os.path.join(path_img, 'Fig7_BEFR_TotalCO2.png'),
                dpi=600, bbox_inches='tight')
    fig.savefig(os.path.join(path_img, 'Fig7_BEFR_TotalCO2.pdf'),
                bbox_inches='tight')

# Relative v. Absolute Decoupling: A Base 100 Analysis

## CO2 emissions: Base 100 Analysis

In [ ]:
# Base 100 = 1980
igu_prod_diff = (100 * be_fr_igu['Production [kt]']
                / be_fr_igu['Production [kt]'].loc[1980]
                )

co2intensity_diff = 100 * co2_intensity / co2_intensity.loc[1980]

co2_abs_igudiff = (100 * be_fr_igu['CO2, IGU production [ktCO2]']
                  / be_fr_igu['CO2, IGU production [ktCO2]'].loc[1980]
                  )

# Plotting production, carbon intensity, CO2 absolute emissions for France
fig, ax = plt.subplots(figsize=(8, 5))
sns.lineplot(data=igu_prod_diff,
             x=igu_prod_diff.index,
             y='France',
             ax=ax)

ax.plot(co2intensity_diff.index,
        co2intensity_diff['co2 interpolated [kg/t]'])
ax.plot(co2_abs_igudiff.index,
        co2_abs_igudiff['France'])

ax.plot([1980, 2015], [100, 100], '--')

ax.set_xlim(1980, 2015)
ax.set_ylim(ymin=0)
sns.despine()
plt.show()

In [ ]:
# Base 100 = 1980
igu_prod_diff = (100 * be_fr_igu['Production [kt]']
                .rolling(5, center=True).mean()
                / be_fr_igu['Production [kt]']
                .rolling(5, center=True).mean()
                .loc[1980]
                )

co2intensity_diff = 100 * co2_intensity / co2_intensity.loc[1980]

co2_abs_igudiff = (100 * be_fr_igu['CO2, IGU production [ktCO2]']
                  .rolling(5, center=True).mean()
                  / be_fr_igu['CO2, IGU production [ktCO2]']
                  .rolling(5, center=True).mean()
                  .loc[1980]
                  )

# Plotting production, carbon intensity, CO2 absolute emissions for France
fig, ax = plt.subplots(figsize=(8, 5))
sns.lineplot(data=igu_prod_diff,
             x=igu_prod_diff.index,
             y='France',
             ax=ax)

ax.plot(co2intensity_diff.index,
        co2intensity_diff['co2 interpolated [kg/t]'])
ax.plot(co2_abs_igudiff.index,
        co2_abs_igudiff['France'])

ax.plot([1980, 2015], [100, 100], '--')

ax.set_xlim(1980, 2015)
ax.set_ylim(ymin=0)
sns.despine()
plt.show()

## Summary

# Dynamic Stock Modelling

Pauliuk and Heeren, + github and notebook

## Define System

In [ ]:
# Create dictionary of model classifications
ModelClassification = {}

# Retrospective MFA from 1945 to 2020
MyYears = list(np.arange(1945, 2020))

In [ ]:
# Classification for time labelled 'Time' must always be present,
# with Items containing a list of odered integers representing years, months,
# or other discrete time intervals
ModelClassification['Time'] = msc.Classification(Name='Time',
                                                 Dimension='Time',
                                                 ID=1, Items=MyYears)

In [ ]:
# Classification for cohort is used to track age-cohorts in the stock.
ModelClassification['Cohort'] = msc.Classification(Name='Age-cohort',
                                                   Dimension='Time',
                                                   ID=1, Items=MyYears)

In [ ]:
# Classification for elements labelled 'Element',
# with Items containing a list of the symbols of the elements covered.
ModelClassification['Element'] = msc.Classification(Name='Elements',
                                                    Dimension='Element',
                                                    ID=2, Items=['IGU'])

In [ ]:
# Classification for regions including Belgium and France.
MyRegions = countries + ['Europe']

ModelClassification['Region'] = msc.Classification(Name='Regions',
                                                   Dimension='Region',
                                                   ID=3, Items=MyRegions)

In [ ]:
# Get model time start, end, and duration:
Model_Time_Start = int(min(ModelClassification['Time'].Items))
Model_Time_End = int(max(ModelClassification['Time'].Items))
Model_Duration = Model_Time_End - Model_Time_Start

In [ ]:
# The indext table lists all aspects needed
# and assigns a classification and index letter to each aspect.
IndexTable = (
    pd.DataFrame({'Aspect': ['Time', 'Age-cohort',
                             'Element', 'Region'],
                  'Description': ['Model aspect "time"',
                                  'Model aspect "age-cohort"',
                                  'Model aspect "Element"',
                                  'Model aspect "Region where flow is"'],
                  'Dimension': ['Time', 'Time', 'Element', 'Region'],
                  'Classification': ([ModelClassification[Aspect]
                                      for Aspect in ['Time',
                                                     'Cohort',
                                                     'Element',
                                                     'Region']]
                                     ),
                  'IndexLetter': ['t', 'c', 'e', 'r']}
                 )
)

# Default indexing of IndexTable:
IndexTable.set_index('Aspect', inplace=True)

IndexTable

In [ ]:
Dyn_MFA_System = msc.MFAsystem(Name='StockAccumulationSystem',
                               Geogr_Scope='3SelectedRegions',
                               Unit='10^6 m²',
                               ProcessList=[],
                               FlowDict={},
                               StockDict={},
                               ParameterDict={},
                               Time_Start=Model_Time_Start,
                               Time_End=Model_Time_End,
                               IndexTable=IndexTable,
                               # Initialize MFA system:
                               Elements=(IndexTable.loc['Element']
                                         .Classification.Items)
                               )

## Creating a DataFrame gathering EU, BE and FR IGU consumption flows

In [ ]:
igu_use = (pd.DataFrame({'Year': list(np.arange(1945, 2021))})
           .set_index('Year'))

In [ ]:
units = ['kt', '10^6 m²']

In [ ]:
# Create a unique DataFrame for EU, FR and BE IGU consumption flows:
for unit in units:
    igu_use[f'EU [{unit}]'] = (eu_igu.interpolate().reset_index()
                               .drop_duplicates(subset=['year'],
                                                keep='last')
                               .set_index(['year'])[f'Consumption [{unit}]'])
    for i in igu_use.index:
        if (i != 2020 and igu_use[f'EU [{unit}]'].isna().loc[i]):
            igu_use[f'EU [{unit}]'].loc[i] = (eu_igu.interpolate().reset_index()
                                              .drop_duplicates(subset=['year'],
                                                               keep='last')
                                              .set_index(['year'])[f'Production [{unit}]']
                                              .loc[i])

In [ ]:
igu_use = pd.concat([igu_use['EU [kt]'], igu_use['EU [10^6 m²]']],
                    axis=1, keys=[('Europe', '[kt]'),
                                  ('Europe', '[10^6 m²]')],
                    names=['Region', 'Unit']).stack()

igu_use = igu_use.unstack()

## Formatting Data

In [ ]:
LifetimeFile = xlrd.open_workbook(
    os.path.join(ROOT_DIR, 'IGU_Lifetime_EU_BE_FR.xlsx'))

Datasheet = LifetimeFile.sheet_by_name('Average_Lifetime')

Lifetimes = []

# Add lifetime values to list
for m in range(1, 4):
    Lifetimes.append(Datasheet.cell_value(m, 1))
print(Lifetimes)

In [ ]:
InflowFile = xlrd.open_workbook(os.path.join(
    DataPath, 'Steel_Consumption_Final_9Countries.xlsx'))
Datasheet = InflowFile.sheet_by_name('FinalSteelConsumption')

# OriginRegion x DestinationRegion x Year
InflowArray = np.zeros((len(MyRegions), len(MyYears)))
print(InflowArray.shape)

OriginCountryPosition = len(MyRegions)
YearPosition = len(MyYears)

for m in range(1, 982):
    InflowArray[0, YearPosition] = Datasheet.cell_value(m, 13)